<font color='tomato'><font color="#CC3D3D"><p>
# DNN Modeling for Multiclass Classification

##### Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import random
from IPython.display import Image, clear_output
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
print(tf.__version__)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

2.12.0


##### Set random seeds to make your results reproducible

In [2]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
reset_seeds(0)

### Step 1: Load and process the data

##### Read data

In [3]:
X_train = pd.read_csv('/kaggle/input/ml-final-exam-type-01/X_train_exam_01.csv', encoding='cp949').drop('custid', axis=1)
y_train = pd.read_csv('/kaggle/input/ml-final-exam-type-01/y_train_exam_01.csv').group
X_test = pd.read_csv('/kaggle/input/ml-final-exam-type-01/X_test_exam_01.csv', encoding='cp949')
IDtest = X_test.ID.unique()
X_test = X_test.drop('ID', axis=1)

X_train.head()

,f000,f001,f002,f003,f004,f005,f006,f007,f008,f009,...,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127
0,0.154932,0.105392,0.068760,0.213738,0.178632,0.185432,0.115931,0.220182,0.049942,0.053519,...,0.129909,0.019110,0.155175,0.118905,0.128475,0.194789,0.250839,0.114440,0.188980,0.124858
1,0.150265,0.113493,0.053723,0.279898,0.220548,0.216846,0.142696,0.249474,0.080952,0.048257,...,0.147738,0.064068,0.207723,0.158806,0.149532,0.215697,0.397651,0.184792,0.182523,0.161461
2,0.163253,0.206024,0.179661,0.253725,0.175406,0.261342,0.212712,0.190733,0.200579,0.076563,...,0.178145,0.087018,0.246528,0.208857,0.079992,0.257902,0.419524,0.318593,0.067054,0.224041
3,0.119036,0.094877,0.061333,0.186340,0.070998,0.166113,0.163255,0.219687,0.068889,0.096517,...,0.057782,0.054187,0.218044,0.099540,0.154490,0.144451,0.194623,0.195095,0.159906,0.086877
4,0.216432,0.172594,0.221798,0.295499,0.245131,0.402211,0.161807,0.313463,0.118416,0.073019,...,0.261979,0.078613,0.318539,0.224083,0.143784,0.372206,0.367924,0.301673,0.187598,0.246487


In [4]:
X_test

,f000,f001,f002,f003,f004,f005,f006,f007,f008,f009,...,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127
0,0.186928,0.236612,0.151582,0.246867,0.188600,0.281135,0.235516,0.335446,0.126411,0.105279,...,0.279986,0.107806,0.261662,0.254477,0.223843,0.314329,0.393487,0.299639,0.230062,0.295807
1,0.226584,0.147807,0.209941,0.334282,0.323352,0.386690,0.249437,0.233099,0.200480,0.097772,...,0.218542,0.095887,0.353289,0.319915,0.150229,0.423384,0.319990,0.436102,0.223772,0.280925
2,0.381486,0.142430,0.201592,0.320655,0.428212,0.281375,0.354500,0.306253,0.114626,0.127123,...,0.232521,0.093212,0.483181,0.191766,0.238918,0.201433,0.487794,0.416235,0.225339,0.188479
3,0.129486,0.047447,0.061566,0.226807,0.126336,0.096881,0.132472,0.195766,0.046641,0.058983,...,0.042702,0.049092,0.166698,0.079480,0.148053,0.128012,0.210556,0.125125,0.173955,0.110259
4,0.168421,0.007434,0.056049,0.185010,0.192001,0.157723,0.131293,0.258516,0.095254,0.086407,...,0.095879,0.073159,0.236936,0.077118,0.187750,0.157764,0.271188,0.281863,0.175666,0.132243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.192915,0.057987,0.113944,0.257024,0.135439,0.091235,0.148286,0.190808,0.034971,0.037820,...,0.076288,0.055579,0.141909,0.091076,0.081312,0.170171,0.209669,0.106173,0.127954,0.143608
14376,0.113116,0.050508,0.048262,0.131850,0.123769,0.093523,0.096673,0.165687,0.051292,0.047523,...,0.054266,0.040469,0.104967,0.063480,0.121991,0.118809,0.189553,0.128352,0.115699,0.088752
14377,0.132465,0.053969,0.064572,0.185991,0.085919,0.102365,0.107560,0.169459,0.056697,0.063935,...,0.072962,0.063038,0.102768,0.101284,0.140798,0.142324,0.198717,0.140842,0.128309,0.096450
14378,0.119209,0.051846,0.085827,0.167591,0.165892,0.108889,0.106144,0.139252,0.040647,0.052591,...,0.051763,0.031795,0.162942,0.136015,0.099129,0.124294,0.172459,0.151070,0.100101,0.110438


##### One-hot-encode Target variable 

In [5]:
# 8개의 범주형 타겟 값을 one-hot-encoding을 통해 8개의 컬럼으로 만들어야 함.
y_train = keras.utils.to_categorical(y_train.astype('category').cat.codes)

##### Split data into train & validation set 

In [6]:
# 학습 데이터를 다시 학습과 검증 데이터로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

##### Feature scaling

In [7]:
# DNN은 scale된 값을 입력값으로 사용해야 함.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Step 2: Define the hyper-model

In [8]:
def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 3, 5)):
        x = keras.layers.Dense(hp.Int('unit_'+str(i), 32, 128, step=32), activation=hp.Choice('activation_'+str(i), ['relu','softmax']))(x)

        x = keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.5, step=0.25, default=0.5))(x)
    outputs = keras.layers.Dense(8, activation='softmax')(x) # 예측값이 8종류이므로 8개 출력 뉴런 필요
    model = keras.Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', # Multiclass Classification에서 사용하는 loss function
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                  metrics=[keras.metrics.CategoricalCrossentropy()]) # Multiclass Classification에서 사용하는 평가지표
    return model

### Step 3: Build multiple hyper-tuned models

In [9]:
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=10,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 08s]
val_categorical_crossentropy: 1.8547368049621582

Best val_categorical_crossentropy So Far: 1.5840495824813843
Total elapsed time: 00h 12m 28s
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Objective(name="val_categorical_crossentropy", direction="min")

Trial 0016 summary
Hyperparameters:
num_layers: 3
unit_0: 96
activation_0: relu
dropout_0: 0.25
unit_1: 128
activation_1: relu
dropout_1: 0.25
unit_2: 32
activation_2: relu
dropout_2: 0.0
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
unit_3: 128
activation_3: relu
dropout_3: 0.0
unit_4: 32
activation_4: softmax
dropout_4: 0.5
Score: 1.5840495824813843


### Step 4: Make submissions

In [10]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest, name="ID"), pred] ,axis=1).to_csv(fname, index=False)


450/450 [==============================] - 1s 2ms/step
'dnn_submission_06200757.csv' is ready to submit.


<font color="#CC3D3D"><p>
# End